In [1]:
# use python3 for this demonstration of ABC
#!/usr/bin/env/python3
# to activate python3 kernel in jupyter, must use the menu "Kernel" to select the
# python3 kernel
# once the kernel is set, the jupyter doc remembers the last kernel activated.
import numpy as np               # for improved math library
import matplotlib.pyplot as plt  # for plotting
from cplane_np import JuliaPlane as jp, JuliaPlaneNV as jnv
%matplotlib inline

# CW-08 Julia Benchmarking (Python)
## README Item #2
## Lance Clifner, Eric Freda
CS-510
<br>October 18, 2016

In [3]:
# run plain vanilla (no numba, non-vectorized code)
%time data = jnv(-2, 2, 5000, -2, 2, 5000, -0.835 - 0.2321j, 100 )


CPU times: user 1min 19s, sys: 324 ms, total: 1min 20s
Wall time: 1min 22s


## Benchmarks
The `refresh()` function is called when the JuliaPlane object is created.  These benchmark numbers are generated from the JuliaPlaneNV class (non-vectorized code) and the JuliaPlane class (modified to support vectorized code).

1. Plain vanilla, 5000x5000 mesh using python for loops for creation of the complex array:
<br>CPU times: user 1min 21s, sys: 588 ms, total: 1min 21s
<br>Wall time: 1min 23s
1. With Numba, 5000x5000 mesh:
<br>CPU times: user 55.7 s, sys: 464 ms, total: 56.2 s
<br>Wall time: 57.2 s
1. With Numba, 5000x5000 mesh using np.linspace/meshgrid creation of the complex array
<br>CPU times: user 1.39 s, sys: 2.31 s, total: 3.7 s
<br>Wall time: 5.55 s

In [2]:
# with numba
%time data = jp(-2, 2, 5000, -2, 2, 5000, -0.835 - 0.2321j, 100 )


CPU times: user 1.46 s, sys: 3.15 s, total: 4.61 s
Wall time: 6.27 s


In [ ]:
# with numba, not vectorized
# (Note to run this cell, you must first comment out the
# @nb.vectorize([nb.int32(nb.complex128)]) line in the cplane_np.py file.
# Then restart the python kernel in jupyter and only then re-run this cell)
%time data = jp(-2, 2, 5000, -2, 2, 5000, -0.835 - 0.2321j, 100 )

## Commentary
It is clear that the plain vanilla, unvectorized code is very slow, taking nearly 20 times longer to run than the vectorized, Just-in-time compiled code of the modified JuliaPlane class.

The speed difference comes from two main aspects:
1. The JuliaPlaneNV class uses two nested python `for` loops to create the individual cells of the rows and columns in the plane, with each cell being passed to the transformation function and returning the final value.  The JuliaPlane class creates the plane using `np.linspace()` and `np.meshgrid()` rather than using `for` loops.
2. The JuliaPlane class uses the `julia()` function which is numba vectorized, meaning that it processes the array quickly rather than cell-by-cell.

We benchmarked the numba modified code using the `for` loops and including numba vectorization of the `julia()` function, and found the execution time to be approximately 1/3 faster than the plain vanilla code.  This shows that the nested python `for` loops are the greatest contributor to the vanilla code's slow performance.  
